# Version 1.0 Model Predictions

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

## Import Data Sets
Here we will import `NetC_Expanded` which is data seperated by management style, result and slavage status, and we will import the predictors and join those datasets.

In [2]:
netc_expanded = pd.read_csv('../Data/NetC_Expanded.csv')
netc_expanded = netc_expanded.drop(['Unnamed: 0'], axis=1)
# netc_expanded = netc_expanded.set_index('Stand_ID')
netc_expanded.head()

,TimeStep,Risk_Cat,Stand_ID,Salvage,Management,Result
0,0,4,0023200606030102900043,True,Heavy,-249.287884
1,0,4,0023200606030102900043,True,NoMgmt,-321.931519
2,0,4,0023200606030102900043,True,Moderate,-276.111511
3,0,4,0023200606030102900043,True,Comm-Ind,-250.583375
4,0,4,0023200606030102900043,True,HighGrade,-293.426896


In [3]:
predictors = pd.read_csv('../Data/Predict_SBW_wCarbon_T0to40.csv')
predictors = predictors.rename(columns={'StandID': 'Stand_ID'})
predictors = predictors.set_index('Stand_ID')
predictors = predictors[["BF_BA","OHost_BA","BF_Stock","OHost_Stock","NonHost_Stock","BF_QMD","ELEV","SLOPE","ASPECT","LAT","SiteInd"]]
predictors.head()

,BF_BA,OHost_BA,BF_Stock,OHost_Stock,NonHost_Stock,BF_QMD,ELEV,SLOPE,ASPECT,LAT,SiteInd
Stand_ID,,,,,,,,,,,
0023200606030200300067,NaN,0.498332,NaN,0.883600,56.9255,NaN,580,5.0,240.0,46.14358,NaN
0023200606030200300826,3.899610,11.890484,21.8437,0.885569,18.6635,4.965398,1170,0.0,0.0,47.19684,28.0
0023200606030200300924,0.036869,7.443510,0.2679,1.166825,3.8019,2.600000,990,0.0,0.0,46.64171,33.0
0023200606030301901813,0.967649,1.368845,63.7216,1.609179,12.1858,2.106063,180,0.0,0.0,45.09319,40.0
0023200606030400901513,3.352901,6.679677,73.4189,1.397641,18.2434,3.405766,250,0.0,0.0,44.73563,NaN


## Selecting Management Style 
Gong to define a management style upfront to create rule set

In [4]:
MANAGEMENT_STYLE = 'Heavy'

In [5]:
mgmt_df = netc_expanded[netc_expanded['Management'] == MANAGEMENT_STYLE]

In [99]:
mgmt_df[mgmt_df['Stand_ID'] == '0023200606030400901513'].groupby('Salvage').agg({'Result':np.mean})

,Result
Salvage,
False,-283.785624
True,-286.491696


Choosing a left join to drop `na` for 

In [6]:
def get_mgmt_df(target_df, pred_df):
    """
    Returns labeled DF for salvage and non salvage decisions
    """
    temp_df = pd.DataFrame(columns=['Stand_ID', 'Salvage_Good', 'Result'])
    for stand in target_df['Stand_ID'].unique():
        group_df = target_df[
            target_df['Stand_ID'] == stand
        ].groupby(['Salvage']).agg({'Result': np.mean})
        
        # If index is True where min is acheived
        if group_df['Result'].idxmin():
            temp_df = temp_df.append(
                pd.DataFrame({
                    'Stand_ID': [stand],
                    'Salvage_Good': [True],
                    'Result': [group_df['Result'].min()]
                })
            )
        else:
            temp_df = temp_df.append(
                pd.DataFrame({
                    'Stand_ID': [stand],
                    'Salvage_Good': [False],
                    'Result': [group_df['Result'].min()]
                })
            )
            
    temp_df = temp_df.set_index('Stand_ID')
    return pd.merge(pred_df, temp_df, on="Stand_ID", right_index=True)

In [7]:
heavy_df = get_mgmt_df(mgmt_df, predictors)

In [8]:
heavy_df = heavy_df.dropna()

In [9]:
heavy_df.head()

,BF_BA,OHost_BA,BF_Stock,OHost_Stock,NonHost_Stock,BF_QMD,ELEV,SLOPE,ASPECT,LAT,SiteInd,Salvage_Good,Result
Stand_ID,,,,,,,,,,,,,
0023200606030200300826,3.899610,11.890484,21.8437,0.885569,18.6635,4.965398,1170,0.0,0.0,47.19684,28.0,False,-334.674834
0023200606030200300924,0.036869,7.443510,0.2679,1.166825,3.8019,2.600000,990,0.0,0.0,46.64171,33.0,True,-298.014370
0023200606030301901813,0.967649,1.368845,63.7216,1.609179,12.1858,2.106063,180,0.0,0.0,45.09319,40.0,True,-384.166289
0023200606030702501209,2.604667,6.283662,46.1259,1.327920,37.9627,3.545086,1360,0.0,0.0,46.33241,31.0,True,-256.942387
0023200606030702501226,17.937933,19.655671,60.2632,0.808564,11.8988,6.622135,1480,25.0,216.0,45.69559,38.0,False,-279.004477


Here we can see that the option to not salvage benefited more than salvaging, so in this case we won't offer salvage credit.

In [143]:
def create_design_matrices(df, quantiles):
    """Create A_N and A_P"""
    measurement = pd.DataFrame()
    measurement['Salvage_Good'] = heavy_df['Salvage_Good']
    labels = ['Q' + str(q) for q in range(1, quantiles+1)]
    
    for col in heavy_df:
        if col == 'Result' or col == 'Salvage_Good':
            continue
            
        try:
            measurement[col] = pd.qcut(heavy_df[col], quantiles, labels=labels)
        except ValueError:
            labels = ['Q' + str(q) for q in range(1, quantiles-1)]
            measurement[col] = pd.qcut(heavy_df[col], quantiles-2, labels=labels)
            labels = ['Q' + str(q) for q in range(1, quantiles)]

    print(measurement.columns)
    measurement = pd.get_dummies(measurement, drop_first=True)
    print(measurement.columns)
    A_p = measurement[measurement['Salvage_Good_True'] == 1].drop('Salvage_Good_True', axis=1)
    A_n = measurement[measurement['Salvage_Good_True'] != 1].drop('Salvage_Good_True', axis=1)
    
    
    return A_p.to_numpy(), A_n.to_numpy(), measurement.to_numpy(), list(A_p.columns)

In [118]:
A_p, A_n, measurement, features = create_design_matrices(heavy_df, 4)

Index(['Salvage_Good', 'BF_BA', 'OHost_BA', 'BF_Stock', 'OHost_Stock',
       'NonHost_Stock', 'BF_QMD', 'ELEV', 'SLOPE', 'ASPECT', 'LAT', 'SiteInd'],
      dtype='object')
Index(['Salvage_Good_True', 'BF_BA_Q2', 'BF_BA_Q3', 'BF_BA_Q4', 'OHost_BA_Q2',
       'OHost_BA_Q3', 'OHost_BA_Q4', 'BF_Stock_Q2', 'BF_Stock_Q3',
       'BF_Stock_Q4', 'OHost_Stock_Q2', 'OHost_Stock_Q3', 'OHost_Stock_Q4',
       'NonHost_Stock_Q2', 'NonHost_Stock_Q3', 'NonHost_Stock_Q4', 'BF_QMD_Q2',
       'BF_QMD_Q3', 'BF_QMD_Q4', 'ELEV_Q2', 'ELEV_Q3', 'ELEV_Q4', 'SLOPE_Q2',
       'ASPECT_Q2', 'LAT_Q2', 'SiteInd_Q2'],
      dtype='object')


In [119]:
pd.DataFrame(A_p, columns=features)

,BF_BA_Q2,BF_BA_Q3,BF_BA_Q4,OHost_BA_Q2,OHost_BA_Q3,OHost_BA_Q4,BF_Stock_Q2,BF_Stock_Q3,BF_Stock_Q4,OHost_Stock_Q2,...,BF_QMD_Q2,BF_QMD_Q3,BF_QMD_Q4,ELEV_Q2,ELEV_Q3,ELEV_Q4,SLOPE_Q2,ASPECT_Q2,LAT_Q2,SiteInd_Q2
0,1,0,1,1,1,0,1,0,1,0,...,0,1,1,1,0,1,1,1,0,1
1,1,1,0,1,1,0,1,1,0,0,...,1,0,1,1,1,0,0,0,0,1
2,1,1,0,1,0,1,1,1,0,1,...,1,0,1,1,1,0,0,1,1,1
3,1,0,1,1,0,1,1,1,0,1,...,1,1,1,1,1,1,0,0,1,1
4,1,0,1,1,0,1,1,1,0,1,...,0,1,1,0,1,1,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1459,1,1,0,1,0,1,1,1,0,0,...,1,0,1,1,1,0,1,1,1,0
1460,1,0,1,0,1,1,1,0,1,1,...,1,1,0,1,1,0,0,0,1,0
1461,1,1,0,1,0,1,1,0,1,0,...,1,1,0,1,1,0,1,1,1,0
1462,1,1,1,1,1,1,0,1,1,1,...,1,1,1,1,0,1,1,1,1,1


### Solve

In [120]:
A_p.shape

(1464, 25)

In [121]:
import gurobipy as gp
from gurobipy import GRB

In [155]:
m = gp.Model("rule-extraciton")

In [156]:
w = m.addMVar(shape=A_p.shape[1], name="weights")
psi_p = m.addMVar(shape=A_p.shape[0], name="psi_p")
psi_n = m.addMVar(shape=A_n.shape[0], name="psi_n")

In [157]:
m.addConstr(w <= 1.0)
m.addConstr(w >= 0.0)
m.addConstr(psi_p <= 1)
m.addConstr(psi_p >= 0)
m.addConstr(psi_n >= 0)
m.addConstr(A_p @ w + psi_p >= 1.0)
m.addConstr(A_n @ w == psi_n)
m.update()

In [158]:
m.setObjective(sum(w) + 100000 * (sum(psi_p) + sum(psi_n)), GRB.MINIMIZE)

In [159]:
m.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (linux64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 9988 rows, 4262 columns and 85527 nonzeros
Model fingerprint: 0x887d14e6
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Presolve removed 8524 rows and 2773 columns
Presolve time: 0.01s
Presolved: 1464 rows, 1489 columns, 27466 nonzeros

Ordering time: 0.00s

Barrier statistics:
 Dense cols : 25
 AA' NZ     : 2.600e+04
 Factor NZ  : 2.779e+04 (roughly 2 MBytes of memory)
 Factor Ops : 5.260e+05 (less than 1 second per iteration)
 Threads    : 1

Barrier performed 0 iterations in 0.02 seconds
Barrier solve interrupted - model solved by another algorithm


Solved with dual simplex
Solved in 1464 iterations and 0.02 seconds
Optim

In [133]:
m.getVarByName("psi_p[0]")
for i in range(measurement.shape[0]):
    print(m.getVarByName("psi_p[" + str(i) + "]"))

<gurobi.Var psi_p[0] (value 1.0)>
<gurobi.Var psi_p[1] (value 1.0)>
<gurobi.Var psi_p[2] (value 1.0)>
<gurobi.Var psi_p[3] (value 1.0)>
<gurobi.Var psi_p[4] (value 1.0)>
<gurobi.Var psi_p[5] (value 1.0)>
<gurobi.Var psi_p[6] (value 1.0)>
<gurobi.Var psi_p[7] (value 1.0)>
<gurobi.Var psi_p[8] (value 1.0)>
<gurobi.Var psi_p[9] (value 1.0)>
<gurobi.Var psi_p[10] (value 1.0)>
<gurobi.Var psi_p[11] (value 1.0)>
<gurobi.Var psi_p[12] (value 1.0)>
<gurobi.Var psi_p[13] (value 1.0)>
<gurobi.Var psi_p[14] (value 1.0)>
<gurobi.Var psi_p[15] (value 1.0)>
<gurobi.Var psi_p[16] (value 1.0)>
<gurobi.Var psi_p[17] (value 1.0)>
<gurobi.Var psi_p[18] (value 1.0)>
<gurobi.Var psi_p[19] (value 1.0)>
<gurobi.Var psi_p[20] (value 1.0)>
<gurobi.Var psi_p[21] (value 1.0)>
<gurobi.Var psi_p[22] (value 1.0)>
<gurobi.Var psi_p[23] (value 1.0)>
<gurobi.Var psi_p[24] (value 1.0)>
<gurobi.Var psi_p[25] (value 1.0)>
<gurobi.Var psi_p[26] (value 1.0)>
<gurobi.Var psi_p[27] (value 1.0)>
<gurobi.Var psi_p[28] (value 1

In [134]:
m.getVarByName("psi_n[0]")
for i in range(measurement.shape[1]):
    print(m.getVarByName("psi_n[" + str(i) + "]"))

<gurobi.Var psi_n[0] (value 0.0)>
<gurobi.Var psi_n[1] (value 0.0)>
<gurobi.Var psi_n[2] (value 0.0)>
<gurobi.Var psi_n[3] (value 0.0)>
<gurobi.Var psi_n[4] (value 0.0)>
<gurobi.Var psi_n[5] (value 0.0)>
<gurobi.Var psi_n[6] (value 0.0)>
<gurobi.Var psi_n[7] (value 0.0)>
<gurobi.Var psi_n[8] (value 0.0)>
<gurobi.Var psi_n[9] (value 0.0)>
<gurobi.Var psi_n[10] (value 0.0)>
<gurobi.Var psi_n[11] (value 0.0)>
<gurobi.Var psi_n[12] (value 0.0)>
<gurobi.Var psi_n[13] (value 0.0)>
<gurobi.Var psi_n[14] (value 0.0)>
<gurobi.Var psi_n[15] (value 0.0)>
<gurobi.Var psi_n[16] (value 0.0)>
<gurobi.Var psi_n[17] (value 0.0)>
<gurobi.Var psi_n[18] (value 0.0)>
<gurobi.Var psi_n[19] (value 0.0)>
<gurobi.Var psi_n[20] (value 0.0)>
<gurobi.Var psi_n[21] (value 0.0)>
<gurobi.Var psi_n[22] (value 0.0)>
<gurobi.Var psi_n[23] (value 0.0)>
<gurobi.Var psi_n[24] (value 0.0)>
<gurobi.Var psi_n[25] (value 0.0)>


In [160]:
m.getVarByName("weights[0]")
for i in range(measurement.shape[1]):
    print(m.getVarByName("weights[" + str(i) + "]"))

<gurobi.Var weights[0] (value 0.0)>
<gurobi.Var weights[1] (value 0.0)>
<gurobi.Var weights[2] (value 0.0)>
<gurobi.Var weights[3] (value 0.0)>
<gurobi.Var weights[4] (value 0.0)>
<gurobi.Var weights[5] (value 0.0)>
<gurobi.Var weights[6] (value 0.0)>
<gurobi.Var weights[7] (value 0.0)>
<gurobi.Var weights[8] (value 0.0)>
<gurobi.Var weights[9] (value 0.0)>
<gurobi.Var weights[10] (value 0.0)>
<gurobi.Var weights[11] (value 0.0)>
<gurobi.Var weights[12] (value 0.0)>
<gurobi.Var weights[13] (value 0.0)>
<gurobi.Var weights[14] (value 0.0)>
<gurobi.Var weights[15] (value 0.0)>
<gurobi.Var weights[16] (value 0.0)>
<gurobi.Var weights[17] (value 0.0)>
<gurobi.Var weights[18] (value 0.0)>
<gurobi.Var weights[19] (value 0.0)>
<gurobi.Var weights[20] (value 0.0)>
<gurobi.Var weights[21] (value 0.0)>
<gurobi.Var weights[22] (value 0.0)>
<gurobi.Var weights[23] (value 0.0)>
<gurobi.Var weights[24] (value 0.0)>
None
